In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import pickle

path_data = 'C:/root/Universita/project_dmdv/data/'

In [6]:
#funzione per estrarre la pagina
#decodificata in utf8, si vede dal tag meta

def get_html_tree(url):
    
    response = requests.get(url)
    html_page = response.content.decode('utf8','replace')#.text
    html_tree = BeautifulSoup(html_page)
    
    return html_tree

In [7]:
#estraggo la pagina base
url_base = 'https://www.goldenglobes.com/winners-nominees/2020'
tree_base = get_html_tree(url_base)

In [9]:
#creo una lista con tutti gli url relativi ad ogni singolo premio
all_tags_gglobeList = tree_base.find_all('ul', class_ = 'dropdown-menu')

#lista di liste da cui costruire il dataset finale
list_of_lists = []

#lista di anni
all_years = [a.text for a in all_tags_gglobeList[0].find_all('a')]
#iterazione su tutti gli anni, per ogni anno si fanno passare tutti i premabsi
for year in all_years:
    print(year)
    #inizializzazione url, pagina di partenza dell'anno
    next_url = url_base.split('/')
    next_url[-1] = year
    next_url = '/'.join(next_url)
    
    #ciclo while finche non ho piu next page
    cond = True
    while cond == True:
        
        tree = get_html_tree(next_url)
        
        #ottengo ogni sezione della pagina (ognuna riferita ad un anno)
        all_groups = tree.find_all('div', class_ = 'view-grouping')
        for current_group in all_groups:
            
            #nome premio
            award = current_group.find('h2', class_ = 'view-grouping-header').div.text
            #tag div: lista di nominati
            all_divs = current_group.find('div', class_ = 'view-grouping-content').findChildren('div', recursive = False)
            
            #iterazione sui nominati
            for current_div in all_divs:
                
                #determino se nominato o vincitore
                is_winner = current_div.find('div', class_ = "views-field views-field-field-nomination-is-winner").text.strip(' ')
                
                #estraggo il nome del titolo e, se presente, il nome della persona
                texts = current_div.find('div', class_ = "views-field views-field-nominee-title").find_all('a')

                #se il premio si riferisce ad una persona, texts e una lista lunga almeno 2 (ultima posizione il nome del titolo),
                #altrimenti 1 in base a quello determino come inserire i dati relativi alle persone e al nome del titolo
                program = texts[-1].text
                person = []
                if len(texts) > 1:
                    person = [t.text for t in texts[:-1]]
                
                if (('ACTOR' in award.upper()) | ('ACTRESS' in award.upper()) | 
                    ('WRITING' in award.upper()) | ('DIRECTING' in award.upper())
                   ) & (len(person) == 0):
                    person = [program]
                    program = None
                
                #aggiungo le informazioni correnti alla lista di liste
                list_of_lists = list_of_lists + [[award, float(year), program, person, is_winner]]
        
        #alla fine del ciclo while
        #estraggo l'url della pagina successiva
        #se c'e solo una pagina associata al premio la linghezza è zero, va fatto quel controllo
        check_next_link = tree.find_all('li', class_ = 'pager-next')
        if len(check_next_link) > 0:
            check_next_link = check_next_link[0].a
        else:
            check_next_link = None
        
        #se none allora ho finito di iterare gli anni
        if check_next_link:
            next_url = 'https://www.goldenglobes.com' + check_next_link['href']
        else:
            next_url = ''
            cond = False

2020
2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
1993
1992
1991
1990
1989
1988
1987
1986
1985
1984
1983
1982
1981
1980
1979
1978
1977
1976
1975
1974
1973
1972
1971
1970
1969
1968
1967
1966
1965
1964
1963
1962
1961
1960
1959
1958
1957
1956
1955
1954
1953
1952
1951
1950
1949
1948
1947
1946
1945
1944


In [10]:
final_data = pd.DataFrame(list_of_lists, columns = ['award', 'year', 'program', 'person', 'is_winner'])
final_data#.head()

,award,year,program,person,is_winner
0,Best Motion Picture - Drama,2020.0,1917,[],Winner
1,Best Motion Picture - Drama,2020.0,"Irishman, The",[],Nominee
2,Best Motion Picture - Drama,2020.0,Joker,[],Nominee
3,Best Motion Picture - Drama,2020.0,Marriage Story,[],Nominee
4,Best Motion Picture - Drama,2020.0,"Two Popes, The",[],Nominee
...,...,...,...,...,...
7985,Best Performance by an Actor in a Supporting R...,1944.0,For Whom The Bell Tolls,[Akim Tamiroff],Winner
7986,Best Director - Motion Picture,1944.0,The Song of Bernadette,[Henry King],Winner
7987,Picture,1944.0,The Song of Bernadette,[],Winner
7988,Actress In A Leading Role,1944.0,The Song of Bernadette,[Jennifer Jones],Winner


In [11]:
final_data.to_pickle(path_data + 'awards/gglobe_data.dataset')